In [2]:
!pip install transformers datasets peft wandb scikit-learn scikit-multilearn
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.1 MB/s eta 0:00:0000:0100:01m
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:0000:0100:01


In [3]:
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from transformers import AutoTokenizer, AutoModel
from datasets import DatasetDict, Dataset
from datasets import Dataset, DatasetDict
from peft import PeftModel, PeftConfig
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

2024-05-19 21:48:10.097462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 21:48:10.097606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 21:48:10.203190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [4]:
# Method and Model Configuration
# ---------------------------------------------------------
entity = "rstern"
debugg = False
model_name = 'filipealmeida/Mistral-7B-v0.1-sharded'
# ---------------------------------------------------------
# emb_org_a, emb_org_b, emb_aug_a, emb_aug_b
experiment_name = "emb_org_a" # model ## augmented_data ## author_label_only
augmented_data = False # True: aug, False: org
author_label_only = False # False: b and True: a
# ---------------------------------------------------------
dataset = 0 # 0, 1 or 2

In [5]:
if dataset == 0:
    test_df = pd.read_csv('/kaggle/input/lp-data/test_df_0.csv')
elif dataset == 1:
    test_df = pd.read_csv('/kaggle/input/lp-data/test_df_1.csv')
else:
    test_df = pd.read_csv('/kaggle/input/lp-data/test_df_2.csv')

# shuffle dataset
test_df = test_df.sample(frac=1, random_state=42)
print(test_df.sample(5))

      Unnamed: 0                                         paragraph1  \
950         9215  >(c)In this section, the term “classified info...   
933         8110  Its authoritarianism by definition. Control fr...   
1357       10024  For anyone wondering why remnants of Nazi symb...   
646         4898  Sure. you can call the cops. They'll take 15 m...   
2127        2426  Putin doesn't care how many of his own country...   

                                             paragraph2  label_author  \
950   It’s all just gross. I’m really sad about the ...             1   
933   Outside of the major urban centers, functional...             1   
1357  Which connections? Most of this stuff is publi...             1   
646   r/politics is currently accepting new moderato...             1   
2127  I don’t know how you know that. Yes, they are ...             1   

      label_dataset  fileindex  
950               0       3505  
933               0       3095  
1357              0       3806  
64

In [6]:
def create_sequences(row):
  sequence = str(row["paragraph1"]) + "[LP2]" + str(row["paragraph2"])
  return sequence

# train_df["input"] = train_df.apply(create_sequences, axis=1)
# val_df["input"] = val_df.apply(create_sequences, axis=1)

def create_multilabel(row):
  label_dataset = max(min(1, row["label_dataset"]), 0)
  multilabel = np.array([int(row["label_author"]), int(label_dataset)])
  return multilabel

def create_singlelabel(row):
  label = np.array([int(row["label_author"])])
  return label

if author_label_only:
  test_df["label"] = test_df.apply(create_singlelabel, axis=1)
  label_weights = [1]
else:
  test_df["label"] = test_df.apply(create_multilabel, axis=1)
  # weight author label heavier than topic change label
  label_weights = [2,1]

y_test = test_df["label"].values
y_test = np.stack(y_test)

x_test_par1 = test_df["paragraph1"].values
x_test_par2 = test_df["paragraph2"].values


In [7]:
ds = DatasetDict({
    'test': Dataset.from_dict({'paragraph1': x_test_par1, 'paragraph2': x_test_par2, 'labels': y_test}),
})

In [8]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('filipealmeida/Mistral-7B-v0.1-sharded')

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
model = AutoModel.from_pretrained('filipealmeida/Mistral-7B-v0.1-sharded', load_in_4bit=True)

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
def last_token_pool(last_hidden_states, attention_mask):
  left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
  if left_padding:
    return last_hidden_states[:, -1]
  else:
    sequence_lengths = attention_mask.sum(dim=1) - 1
    batch_size = last_hidden_states.shape[0]
    return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


def collate_fn(batch):
    max_length = 4096
    embeddings1_list = []
    embeddings2_list = []
    labels_list = []

    for sample in batch:
        paragraph1 = sample['paragraph1']
        paragraph2 = sample['paragraph2']
        labels = sample['labels']
        labels_list.append(labels)

        inputs1 = tokenizer(paragraph1, max_length=max_length, truncation=True, return_tensors='pt')
        inputs2 = tokenizer(paragraph2, max_length=max_length, truncation=True, return_tensors='pt')

        with torch.no_grad():
            outputs1 = model(**inputs1)
            embeddings1 = last_token_pool(outputs1.last_hidden_state, inputs1['attention_mask'])

            outputs2 = model(**inputs2)
            embeddings2 = last_token_pool(outputs2.last_hidden_state, inputs2['attention_mask'])

        embeddings1_list.append(embeddings1.float())
        embeddings2_list.append(embeddings2.float())

    embeddings1_batch = torch.stack(embeddings1_list, dim=0)
    embeddings2_batch = torch.stack(embeddings2_list, dim=0)
    labels_batch = torch.tensor(labels_list)

    return embeddings1_batch, embeddings2_batch, labels_batch

In [16]:
# Define the model architecture for the classifier
class Classifier(nn.Module):
    def __init__(self, input_size, number_labels=1):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size * 2, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, number_labels)  # Assuming binary classification

    def forward(self, emb1, emb2):
        # Concatenate embeddings along the last dimension
        x = torch.cat((emb1, emb2), dim=2)  # Shape: [batch_size, input_size * 2]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# define params
if author_label_only:
  number_labels = 1  # or 2 for binary classification with two labels
else:
  number_labels = 2
batch_size = 16
input_size = 4096 # needs to be set as well above
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = nn.BCEWithLogitsLoss()

model_save_path = '/kaggle/input/lp-test-model/model.pth'

In [17]:
# Instantiate the model
classifier = Classifier(input_size, number_labels)

# Load the saved model weights
classifier.load_state_dict(torch.load(model_save_path))

# Ensure the model is in evaluation mode
classifier.eval()

# Move the model to the appropriate device
classifier.to(device)

# get DataLoader
test_loader = DataLoader(ds['test'], batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
# List to store predictions and true labels
all_predictions = []
all_labels = []
losses = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch_embeddings1, batch_embeddings2, target in test_loader:
        batch_embeddings1 = batch_embeddings1.to(device).float()
        batch_embeddings2 = batch_embeddings2.to(device).float()
        target = target.to(device).float()

        # Forward pass
        output = classifier(batch_embeddings1, batch_embeddings2)

        # Calculate the loss
        if number_labels == 2:
            loss = loss_function(output.squeeze(), target.float())
        else:
            loss = loss_function(output.squeeze(), target.squeeze().float())
            losses.append(val_loss.item())
        
        # Apply sigmoid to get probabilities, then round to get binary predictions
        # Calculate predictions
        predictions = torch.sigmoid(output).round().squeeze()
        all_predictions.extend(predictions.cpu().tolist())
        all_labels.extend(target.cpu().tolist())
        
        # convert predictions and target to right format
        predictions = torch.sigmoid(output).round().squeeze().cpu().tolist()
        target = target.cpu().tolist()
        
        # calculate scores
        train_f1_macro = f1_score(target, predictions, average="macro")
        
        if number_labels == 2:
            f1_macro_author = f1_score([[row[0]] for row in target], [[row[0]] for row in predictions])
            print({"loss": loss, "train_macro_f1": train_f1_macro, "train_macro_f1_author": f1_macro_author})
        else:
            print({"loss": loss, "train_macro_f1": train_f1_macro})

print("Overall Results")

# Calculate validation metrics
val_f1_macro = f1_score(val_targets, val_predictions, average="macro")
val_f1_micro = f1_score(val_targets, val_predictions, average="micro")
scores = {'epoch': epoch+1, 'loss': loss.item(), 'val_f1_macro': val_f1_macro, 'val_f1_micro': val_f1_micro}
if number_labels == 2:
    f1_macro_author = f1_score([[row[0]] for row in val_targets], [[row[0]] for row in val_predictions])
    scores['val_f1_author'] = f1_macro_author
    # Log validation metrics to wandb
print(scores)

/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


{'loss': tensor(1.0749, device='cuda:0'), 'train_macro_f1': 0.23809523809523808, 'train_macro_f1_author': 0.47619047619047616}
{'loss': tensor(1.1980, device='cuda:0'), 'train_macro_f1': 0.15789473684210528, 'train_macro_f1_author': 0.31578947368421056}
{'loss': tensor(0.8904, device='cuda:0'), 'train_macro_f1': 0.3181818181818182, 'train_macro_f1_author': 0.6363636363636364}
{'loss': tensor(0.8606, device='cuda:0'), 'train_macro_f1': 0.41666666666666663, 'train_macro_f1_author': 0.8333333333333333}
